# Optical Character Recognition using Python

## - Sheffield Ambassadors, 11th May 2016 -

Optical Character Recognition (OCR) systems convert images of text into computer-readable text files, i.e. the image of each character is converted into its corresponding ASCII or Unicode character code. This is a classification problem: the computer is given some image data and has to choose the correct character code class. Classification is one of the central problems in machine learning and artificial intelligence.

In this session you will be experimenting with some simple OCR tools written in Python.

By the end of this session you will understand,

* how a basic OCR system works,
* how to evaluate an OCR system,
* some of the challenges of getting OCR to work reliably,
* how to use Python in the iPython Notebook environment.

The session ends with an online OCR challenge in which you can participate.


## Preliminary note: Using iPython notebook
This session is using the iPython notebook. The notebook provides a simple way for mixing notes and code. The notebook is composed of 'cells'. Some cells contain text (like this one), and others contain bits of Python code that you can run. 

To run a cell,
* first click on it so that it is in focus (a border will appear around the cell) and then,
* either click on the play button in the tool bar or press shift-enter on the keyboard. 

Try this now by executing the 'Hello World' cell below.

In [ ]:
print('Hello World!')

##Step 1: Setting Up

Before we start we must set up our environment by loading in some Python modules that we are going to use later. 

Read the code in the next cell and then run it.

In [ ]:
# Setting Up
# This cell loads in some modules that we wil be using later

# Load standard modules for matrix maths, plotting and GUI elements
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
%matplotlib inline
from ipywidgets import interact, interactive, fixed
from IPython.display import Audio, display

# Load the OCR module that was written for this session
import python_ocr as po
from python_ocr.api import submit

print('Set up successful')

## Step 2 - Loading the data

### About the data

This lab class is going to use some character image data that we have prepared earlier. The data has been split into two sets: training data and testing data. The OCR system will use the training data to learn what characters look like; the testing data will be used to test how well the OCR is working.

The images were preprocessed by resizing each character image to be exactly 30 by 30 pixels. The brightness values of these pixels range from 0 (black) to 767 (white). These 30 x 30 brightness values are then stored in a long row of 900 numbers, i.e. there are 900 numbers being used to represent each image.

In the training data there are 699 characters. The data is stored in a big grid ('a matrix') each row of the grid represents one character (i.e. each row has 900 numbers). There are 699 rows in the grid to represent the full set of 699 characters.  The testing data is stored in the same way but there are only 200 test characters and so only 200 rows in the test grid (matrix).

The training data and test data come with labels to tell you what the correct answers are. The labels are integers from 1 to 26 where 1 means 'A', 2 means 'B' etc. There is a list of labels for the training examples (i.e. containing 699 labels) and a separate list of labels for the test examples (containing 200 labels).

### Loading the data

The data can be loaded by using a pair of functions that have been provided in the python_ocr module: load_train and load_test.

Execute the cell below to load all the data,


In [ ]:
# This cell loads in the image data that we will use
train_data, train_labels = po.load_train()
test_data, test_labels = po.load_test()
print('Data Loaded')

### Checking the data

The training characters are stored as the rows of the matrix train_data. In Python we can handle matrixes using a package called numpy. Using numpy we can extract the (n+1)th row of the train_data matrix by simply writing train_data[n,:]. For example to get the 11th row,

    row = train_data[10, :]

The variable *row* will now store the list of 900 pixel values for the 10th training character. The python_ocr module provides a function called display_character that will display these 900 pixel as a 30 by 30 pixel image, 

    po.display_character(row)

Try typing the two lines above into the cell below and then executing the cell.

In [ ]:
# Add code here



If the data loaded correctly you should see the 11th letter of the training data set which happens to be an 'H'. 

Try selecting another row by changing the 10 in your previous cell to another integer in the range 0 to 698. Then re-run the cell.

##Step 3 - Adding noise to the data

The training and test data comes from images of printed text. The images are all of quite high quality. The classifier that we will be using will recognise most of the data correctly, i.e. the task is a bit too easy. To make it more challenging we will corrupt the test images by adding noise, i.e. changing the pixel values by random amounts.

### Interactive Notebook elements

Before proceeding we will introduce a useful iPython notebook tool. The notebook provides some simple user interface elements, e.g. sliders. Sliders allow you to interactively control the parameter value of a function. Sliders are added by using a function called 'interactive'. The first parameter passed to interactive is the name of the function you wish to control. The second parameter is a tuple that gives the range of values for the slider.

For example, say that you have a function called, draw_square(n) that draws a square of size n. We could then make a slider that would allow us to draw squares of any size between 0 and 100 by doing,

    interactive(draw_square, (0, 100))

In the cell below I have written a function called display_character(n) than displayss the nth character from the training data. I've then used interactive, to add a slider that selects which character to display. 

Execute the cell below and prepare to be amazed... :)


In [ ]:
def display_character(n):
    # A display function that 'interactive' can call
    po.display_character(train_data[n,:])

# The call to 'interactive' that produces the slider
interactive(display_character, n=(0,100))

### Adding noise

'Adding noise' to an image simply means to perturb the image values by random amounts. To add noise to a character from the test data we will first generate 900 random numbers in the range 0 to 1.  We will then multiply them by a noise level value. For example, if the noise level is 100 then the random numbers will be multiplied by 100 to be in the range 0 to 100. Finally we add the noise onto the 900 pixel values belonging to the test character.

The cell below provides an interactive demonstration of the effect of adding noise. Try it now.

In [ ]:
def display_noisy_character(letter_index, noise_level):
    # Make 900 random numbers in the range 0 to 1
    noise = np.random.random_sample((1, 900))
    # Add the noise to the nth letter
    nth_letter = test_data[letter_index, :]
    noisy_letter = nth_letter + noise * noise_level
    # Display the noise result
    po.display_character(noisy_letter)
    
# Two sliders: one to select the character; one to set the noise level
interactive(display_noisy_character, letter_index=(0,199), noise_level=(0,300))

The interactive demonstration above added noise to a single character at a time. We can use the same idea to add noise to all the test characters in one go

    noise_level = 200
    noise = np.random.random_sample((200, 900))
    noisy_test_data = test_data + noise_level * noise

Copy the above code into the cell below and then execute it. (Note, you will need to remove the indentation). We will use noisy_test_data later.

In [ ]:
# Add code here
noise_level = 200
noise = np.random.random_sample((200, 900))
noisy_test_data = test_data + noise_level * noise

## Step 4 - Classifying the letters

We are now going to try classifying the letters. The python_ocr module contains a function called classify which implements a 'nearest neighbour' classifier. This is a simple but powerful classifier. It compares the image to be classified against all the examples in the training data and finds the example that it matches most closely. It then simply returns the label of that example. 

Aside: What does 'matches most closely' mean? Well, the difference between a pair of images can be measured in many different ways. One way would be to subtract one image from the other and measure the average absolute size of the difference. This does not work well if there is a difference in the lighting, i.e. if the test images are brighter or darker than the training images. The code provide uses something called the 'cosine distance' which is not sensitive to the lighting variation. 


### Preparation

The labels and the classifier outputs are numbers 1 to 26. To make the results easier to see we will write a little function to turn these into the characters 'A' to 'Z'.

Execute the cell below to define and test the label_to_letter function

In [ ]:
def label_to_letter(label):
    # Turn a label into a letter
    return chr(ord('A') - 1 + label)

# Test
print('1 means ' + label_to_letter(1))
print('26 means ' + label_to_letter(26))

### Classifying a character

We are now ready to test the classifier. Read the code in the cell below and then execute it.

In [ ]:
n = 10
nth_character = test_data[n,:]
guessed_label = po.classify(train_data, train_labels, nth_character)
true_label = test_labels[n]
print("The true label is " + label_to_letter(true_label))
print("The classifier thinks it is " + label_to_letter(guessed_label[0]))
po.display_character(nth_character)

We are now going to make it more interesting by wrapping the classifier in a little function that can be attached to a slider and called interactively.

Run the cell below and using the slider see if you can find any examples that are recognised incorrectly.

In [ ]:
def classify_character(n):
    nth_character = test_data[n,:]
    true_label = test_labels[n]
    guessed_label = po.classify(train_data, train_labels, nth_character)
    print("The true label is " + label_to_letter(true_label))
    print("The classifier thinks it is " + label_to_letter(guessed_label[0]))
    po.display_character(nth_character)

          
interactive(classify_character, n=(0,199))

Could you find any characters that were recognised incorrectly? What mistakes were made?

### Classifying noisy characters

You will have seen that the classifier makes very few mistakes. Let's now repeat the demonstration but this time with noise added to the images. 

Examine the code in the cell below and then run it.

In [ ]:
def classify_noisy_character(n, noise_level):
    noise = np.random.random_sample(train_data[n,:].shape)
    noisy_letter = test_data[n, :] + noise_level * noise;
    true_label = test_labels[n]
    guessed_label = po.classify(train_data, train_labels, noisy_letter)
    print("The noise level is " + str(noise_level))
    print("The true label is " + label_to_letter(true_label))
    print("The classifier thinks it is " + label_to_letter(guessed_label[0]))
    po.display_character(noisy_letter)

          
interactive(classify_noisy_character, n=(0,100), noise_level=(0,700))

# Step 5 - Evaluating the classifier

So far we have been classifying one character at a time and comparing the output with the correct label by eye. This has given us a rough idea about how well the classifier works but we'd like to be able to evaluate the classifier precisely. 

To precisel evaluate the classifier we will use it to classify all 200 characters in the test set and then we will simply count what percentage are recognised correctly.

### Computing the 'percentage correct' score

In the previous sections we passed a single row of test_data to the classify function and it returned a single label. However, the classify function has been written so that it can classify many characters at once and return a list of labels. To do this we simply have to pass it multiple images stored in multiple rows of a matrix. In fact, we can pass the complete test_data matrix in one go at follows,

    guessed_labels = po.classify(train_data, train_labels, test_data)
    print(guessed_labels)
    
 Try this in the cell below,

In [ ]:
# Add code here


Now to compute the score we simply have to count how many times the number in the list guessed_labels matches the corresponding number in the correct label list, test_labels. guessed_labels and test_labels are actually Numpy arrays. The elements can be compared very easily using,

    test_labels == guessed_labels
   
This will produce a list of boolean values, i.e.,trues and falses. We can then count the number of trues by doing,
 
    np.sum(test_labels == guessed_labels)

There are 200 characters being tested so to compute the percentage that are correct we can write,

    percentage_correct = np.sum(test_labels == guessed_labels) / 200.0  * 100.0
    print(percentage_correct)
   
Execute these last two lines in the cell below. Remeber, 100% means every character was recognised correctly; 0% would mean that every character was recognised incorrectly.

In [ ]:
# Add code here


### Testing robustness to noise

Now repeat the classification and evaluation steps using the noisy data that you made earlier -- it was called noisy_test_data. Has the performance got worse?

Use the cell below

In [ ]:
# Add code here
guessed_labels = po.classify(train_data, train_labels, noisy_test_data)
percentage_correct = np.sum(test_labels == guessed_labels) / 200.0  * 100.0
print(percentage_correct)


Look back at the code that you used to generate the noisy data. By adapting this code make a new set of noisy test data with a greater amount of noise (i.e. increasing the value of noise_level). How big do you have to make noise level before the classifier starts getting more characters wrong than right? i.e. a result of less than 50%. 

Answer this question by writing code in the cell below.

(Advanced version! You could do this using a slider to set the noise level)

In [ ]:
# Add code here
def classify_noisy(noise_level):
    noise = np.random.random_sample((200, 900))
    noisy_test_data = test_data + noise_level * noise
    guessed_labels = po.classify(train_data, train_labels, noisy_test_data)
    percentage_correct = np.sum(test_labels == guessed_labels) / 200.0  * 100.0
    print(percentage_correct)


interactive(classify_noisy, noise_level=(0,1200))

## Step 6 - Analysing the classifier errors

### The Confusion Matrix

The percentage correct score is a measure of the overall classifier performance but it doesn't help us understand the mistakes that are made. A useful tool for understanding the mistakes is a 'confusion matrix'. A confusion matrix is a representation of the classifier's output. Our confusion matrix will have 26 rows and 26 columns. The rows of the matrix represent the correct labels (1 to 26) and the columns represent the classifier outputs (1 to 26). The contents of a cell show how many times an particular input led to a particular output. For examples if the matrix has 5 in the row 4 and column 10 that would mean that class 4 ('D') was recognised as class 10 ('J') 5 times. Alternatively, if there is a 12 in row 4 and column 4 that would mean that class 4 was recognised as class 4 ('D') 12 times, i.e. 12 D's were recognised correctly. If a classifier makes no mistakes then the confusion matrix will have zeros everywhere except along it's diagonal.

The python_ocr module has a function called evaluate. This function takes the list of correct labels and the list of classifer output labels as input. It then returns both the percentage correct and a confusion matrix. This is demonstrated in the next cell which uses the matshow function to show display the confusion matrix,

In [ ]:
score, confusion_matrix = po.evaluate(test_labels, guessed_labels)
print(score)
plt.matshow(confusion_matrix, cmap=cm.gray_r)

Notice that most cells of the matrix are 0 (white) apart from along the diagonal. This means the classifer is working well because the entries along the diagonal are where the classifier has made a correct decision. 

### Classifying distorted characters

We have tried adding noise to the characters, now lets see what happens if we distort the letters in different ways. We will try two different things,

    i/ reflecting the letter upside down
    ii/ inverting the black/white values.

Functions to perform these transformations have been provided

    reflected_test_data = po.reflect_data(test_data)
    
    inverted_test_data = po.invert_data(test_data)

Type the lines of code in the cell below and execute it

In [ ]:
# Add code here
inverted_test_data = po.invert_data(test_data)
reflected_test_data = po.reflect_data(test_data)


Use display_character to display one of the reflected letters to check that it has worled

In [ ]:
#Add code here
po.display_character(reflected_test_data[4, :])

Now run the and evaluate the classifier on each set of distorted letters. For each set, display the corresponding confusion matrix and print the score.

In [ ]:
# Evaluate using the inverted_test_data
# Add code here
guessed_labels = po.classify(train_data, train_labels, inverted_test_data)
score, confusion_matrix = po.evaluate(test_labels, guessed_labels)
plt.matshow(confusion_matrix, cmap=cm.gray_r)
print(score)

In [ ]:
# Evaluate using the reflected_test_data
# Add code here
guessed_labels = po.classify(train_data, train_labels, reflected_test_data)
score, confusion_matrix = po.evaluate(test_labels, guessed_labels)
plt.matshow(confusion_matrix, cmap=cm.gray_r)
print(score)

What do you notice about the confusion matrices. Do they look the same? Are any of the letters in the reflected_test_data classified correctly. Does this make sense?

# Step 7 - Improving the classifier

We saw in the last section that our OCR system worked very poorly when we applied some simple distortions to the characters. For example, if we turned a character upside down then the classifier could no longer recognise it. Humans don't have the problem. We have learnt what characters look like when you turn them upside down. So lets now teach our OCR system what upside down characters look like. We can do this by extending the training data.

We will make inverted and reflected versions of the training data. We will then join all the different versions of the training data together using numpy's vstack command which stacks matrices on top of each other. (We will also have to copy the training data labels.) 

Read the code below carefully before executing the cell.

In [ ]:
# Make distorted versions of the training data
reflected_train_data = po.reflect_data(train_data)
inverted_train_data = po.invert_data(train_data)
# Combined the distorted versions and the original training data using vstack (vertical stack)
all_train_data = np.vstack((train_data, reflected_train_data, inverted_train_data))
# Copy the labels three times
all_train_labels = np.hstack((train_labels, train_labels, train_labels))

Now that we've prepared the training data we can use it in the usual way by passing it to the classify function.

    guessed_labels = po.classify(all_train_data, all_train_labels, test_data)

Use po.classify and po.evaluate to compute scores using the new training data for the 
inverted test data, reflected test data and the original test data. What do you notice about the new scores.

In [ ]:
# Add code here
guessed_labels = po.classify(all_train_data, all_train_labels, test_data)
score, cm = po.evaluate(test_labels, guessed_labels)
print(score)
guessed_labels = po.classify(all_train_data, all_train_labels, inverted_test_data)
score, cm = po.evaluate(test_labels, guessed_labels)
print(score)
guessed_labels = po.classify(all_train_data, all_train_labels, reflected_test_data)
score, cm = po.evaluate(test_labels, guessed_labels)
print(score)

Has the extended training data set improved the classifier performance? 

# Step 8 - Classifier Challenge

For this final step you are presented with a character classifcation challenge. 

We have provided some characters that have been distorted in various different ways. Your challenge is to try to correctly recognise as many of the characters as possible. You have to use the classify algorithm provided, but you can choose what to use as training data.

First of all we will load in the challenge data. Execute the cell below.


In [ ]:
challenge_data = po.load_challenge()

Notice that there are no correct labels supplied this time. If we'd given you the correct labels it would be too easy to cheat!

We can now view the data by using display_character and the interactive slider element that we used earlier. You will see that a variety of distortions have been applied and some of the character are hard to recognise even for humans,

In [ ]:
def display_character(n):
    # A display function that 'interactive' can call
    po.display_character(challenge_data[n,:])

# The call to 'interactive' that produces the slider
interactive(display_character, n=(0,100))

Now let's try running the classifier using the standard training data set

In [ ]:
guessed_labels = po.classify(train_data, train_labels, challenge_data)
print(guessed_labels)

We have output the guessed labels but without knowledge of the correct labels how we see how well the classifier has performed?

To solve this problem we have provided a remote evaluation service running on a server on the internet. This server is storing the correct labels and can therefore compute your score. To get your score you send your guessed labels to the server and it will compute your result and post it on a challenge website.

We have written a function called 'submit' that will send your results to the server. The submit is used like this,

    submit(username='myname', password='secretword', trialname='test1', trialdata=guessed_labels)
    
When you use it set username to your own name -- or a nickname -- and make up any password and any trialname, e.g. 
 
    submit(username='jon', password='aardvark76', trialname='baseline', trialdata=guessed_labels)
    
Try out the submit function in the cell below. Try submitting the guessed_labels computed above,

In [ ]:
# Add your submit instruction here
submit(username='jon', password='password', trialname='baseline', trialdata=guessed_labels)

To view your submission on the server click on this link http://178.62.73.32 to get to the challenge website. Log in using the same user name and password that you used in the submit function.

Your challenge now is to make a new version of train_data and train_labels that is able to produce a better classification result. You can start by inverting and reflecting the data in the same way that you did earlier. However, to get a really good score you will need to add further distortions that mimick the distortions that you can see in the test data. 

Good luck!

In [ ]:
# Build your training data and run the classifier in this cell
#
#